In [244]:
import csv
import time
import ast
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from zawar_keys import tapi_key
from zawar_functions import *

### Getting Top Rated Movies from TMDB API

In [66]:
with open("TMDB_toprated.csv", "w", newline="", encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows([['id','title','original_language','avg_vote','vote_count','popularity','genres','release_date','adult']])

In [67]:
p=1
while p<=357:
    url_params['page']=p
    
    results=get_tmdb_toprated(url_params,tapi_key)
    
    parsed_results=parse_toprated(results)
    
    save_data("TMDB_toprated.csv",parsed_results)
    p+=1

### Getting Movie Details

In [72]:
toprated_df=pd.read_csv('TMDB_toprated.csv')
mov_ids=toprated_df['id'].values
mov_ids

array([19404,   278,   238, ...,  5491, 11059, 14164], dtype=int64)

In [78]:
with open("TMDB_movie_details.csv", "w", newline="", encoding='utf-8') as m:
    writer = csv.writer(m)
    writer.writerows([['id','imdb_id','budget','revenue','genre_names','original_title','production_countries',
                       'runtime','status']])

In [88]:
for mov_id in mov_ids[4996:]:
    mov_info=get_movie_details(mov_id,tapi_key)
    
    parsed_mov=[parse_movie(mov_info)]
    
    save_data("TMDB_movie_details.csv", parsed_mov)

### Webscraping Cumulative Worldwide Revenue for Movies from IMDB Website:

In [237]:
df3= pd.read_csv('clean_df.csv')
df3.shape

(1015, 16)

In [234]:
imdb_ids=df3['imdb_id'].values

In [239]:

cum_revenues=[]
for imdb_id in imdb_ids:
    imdb_url='https://www.imdb.com/title/{}'.format(imdb_id)
    page=requests.get(imdb_url)
    soup=BeautifulSoup(page.text,'html.parser')
    details=soup.find('div', id='main_bottom').find('div', id='titleDetails')
    l=details.find_all('div',class_='txt-block')
    n=np.nan
    for x in l:
        
        try:
            if x.h4.text=='Cumulative Worldwide Gross:':
                n=x.contents[2].strip()
        except:
            continue
    cum_revenues.append([imdb_id,n])
    time.sleep(.2)

In [242]:
with open("cumulative_revenue.csv", "w", newline="", encoding='utf-8') as y:
    writer = csv.writer(y)
    writer.writerows([['imdb_id','cumulative_revenue']])

In [243]:
save_data('cumulative_revenue.csv', cum_revenues)